In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import warnings
import pickle
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA


In [2]:
warnings.filterwarnings("ignore")

In [3]:
#SELECTKBEST FUNCTION
def princomp(indep_x,dep_y,n):#Create a function princomp with 3 parameter
   pca = PCA(n_components=n)#select best feature using PCA
   pca_fit=pca.fit(indep_x)#create a model based on best feature
   pca_features=pca_fit.transform(indep_x)#Create a new indep_x dataset based on k values
   #explained_variance=pca.explained_variance_ratio_  
   return pca_features    
    

In [4]:
#STANDARDIZATION FUNCTION
def split_scalar(indep_x,dep_y):
      X_train, X_test, y_train, y_test = train_test_split(indep_x, dep_y, test_size = 0.25, random_state = 0)#split training set and test set
      sc = StandardScaler() # call standardization function
      X_train = sc.fit_transform(X_train)#find mean and standard divation for x_train
      X_test = sc.transform(X_test)    
      return X_train, X_test, y_train, y_test
    

In [5]:
#FIND EVALUATION METRICS FOR REGRESSION
def r2_prediction(regressor,X_test,y_test):#Create a function r2_prediction with 3 arguments
     y_pred = regressor.predict(X_test)#findout the predicted output based on X_test(input)
     r2=r2_score(y_test,y_pred)#Findout r2_score value 
     return r2

In [6]:
#REGRESSION ALGORITHM
def Linear(X_train,y_train,X_test):       
        regressor = LinearRegression()#Call linearregression fun
        regressor.fit(X_train, y_train)#Create a model
        r2=r2_prediction(regressor,X_test,y_test)#call r2_prediction & findout r2 value 
        return  r2   

def svm_linear(X_train,y_train,X_test):
        regressor = SVR(kernel = 'linear')
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2  

def svm_NL(X_train,y_train,X_test):   
        regressor = SVR(kernel = 'rbf')
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2    
    
def Decision(X_train,y_train,X_test):
        regressor = DecisionTreeRegressor(random_state = 0)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2 
    
def random(X_train,y_train,X_test):       
        regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2 
    
    

In [7]:
#CREATE A TABLE
def pca_regression(acclin,accsvml,accsvmnl,accdes,accrf): #Create a function  with 5 arguments
    #Create a table,row named as Chisquare,column named as linear,SVMl,SVMnl,Decision,Random  
    dataframe=pd.DataFrame(index=['PCA'],columns=['Linear','SVMl','SVMnl','Decision','Random'])                                                                                     
    for number,idex in enumerate(dataframe.index):
        dataframe['Linear'][idex]=acclin[number] # #dataframe['Linear'][PCA]=r2lin[0]    
        dataframe['SVMl'][idex]=accsvml[number]#dataframe[SVMl'][PCA]=r2svml[1]
        dataframe['SVMnl'][idex]=accsvmnl[number]#dataframe['SVMnl'][PCA]=r2svmnl[2]
        dataframe['Decision'][idex]=accdes[number]#dataframe['Decision'][PCA]=r2des[3]
        dataframe['Random'][idex]=accrf[number]#dataframe['r2rf'][PCA]=r2rf[4]
    return dataframe

In [8]:
 
dataset1=pd.read_csv("prep.csv",index_col=None)#read the csv file,index_col=None means index number starts from 0,1...
df2=dataset1#Assign dataset1 into df2
df2 =pd.get_dummies(df2, drop_first=True)#Convert categorical data into numercial data,drop_first=true means drop on dummy column.
indep_x=df2.drop('classification_yes', axis=1)#drop classification_yes,1 means drop column wise
dep_y=df2['classification_yes']#Select classification_yes from df2 and stored it dep_y 


In [13]:

# call the fun & put k=5    
principle=princomp(indep_x,dep_y,6) 
acclin=[]#Create a empty list
accsvml=[] 
accsvmnl=[]
accdes=[]
accrf=[]

X_train, X_test, y_train, y_test=split_scalar(principle,dep_y) #Kbest select best 5 feature,and split into training set and test set    
for i in principle:   
    r2_lin=Linear(X_train,y_train,X_test)
    acclin.append(r2_lin)
    
    r2_sl=svm_linear(X_train,y_train,X_test)    
    accsvml.append(r2_sl)
    
    r2_NL=svm_NL(X_train,y_train,X_test)
    accsvmnl.append(r2_NL)
    
    r2_d=Decision(X_train,y_train,X_test)
    accdes.append(r2_d)
    
    r2_r=random(X_train,y_train,X_test)
    accrf.append(r2_r)
    
    
result=pca_regression(acclin,accsvml,accsvmnl,accdes,accrf)#call selectk_regression fun then only the output comes in table format


In [14]:
result


,Linear,SVMl,SVMnl,Decision,Random
PCA,0.389481,0.357726,0.612626,0.392361,0.633247
